<a href="https://colab.research.google.com/github/Nutsudapenpong/DPDM/blob/master/Followmalaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 import numpy as np
 import time
 import sys
 import os
 import random
 #from skimage import io
 import pandas as pd 
 from matplotlib import pyplot as plt
 from shutil import copyfile

 import cv2
 import tensorflow as tf

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
base_path = '/content/drive/My Drive/Project/malaria' 

train_path =  '/content/drive/My Drive/Project/malaria/training.json'

In [5]:
import json
import csv

#read file
with open(train_path, 'r') as myfile:
  data=myfile.read()

#parse file
objlist = json.loads(data)
_list = []
for obj in objlist:
  _imageRow = []
  img = obj['image']
  _imageRow.append(img['checksum'])
  _imageRow.append(img['pathname'])
  shp = img['shape']
  _imageRow.append(shp['r'])
  _imageRow.append(shp['c'])
  _imageRow.append(shp['channels'])

  bound0bjs = obj['objects']
  for bound0bj in bound0bjs:
    _bondBox = []

    bbox = bound0bj['bounding_box']
    bmin = bbox['minimum']
    _bondBox.append(bmin['r'])
    _bondBox.append(bmin['c'])

    bmax = bbox['maximum']
    _bondBox.append(bmax['r'])
    _bondBox.append(bmax['c'])
    _bondBox.append(bound0bj['category'])

    #Prepare record
    _newRow = []
    for r in _imageRow:
      _newRow.append(r)
    for r in _bondBox:
      _newRow.append(r)
    _bondBox.remove
    _list.append(_newRow)
  _imageRow.remove
#print(_list)
with open('train.csv', 'w', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(['checksum', 'pathname', 'shapeR', 'shapeC', 'channels', 'bbminR', 'bbminC', 'bbmaxR', 'bbmaxC', 'category'])
  writer.writerows(_list)